<a href="https://colab.research.google.com/github/tigerywh/colab/blob/master/langchain_chatglm_ddup2_share.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

基于chatGLM+chain构建图书问答机器人

以《动手学深度学习（Dive into Deep Learning，D2L.ai）》书籍 PDF 版本为例

# 参考：

https://github.com/d2l-ai/d2l-zh/releases

https://github.com/imClumsyPanda/langchain-ChatGLM.git

In [ ]:
import locale 
locale.setlocale(locale.LC_ALL, 'en_US.UTF-8')

'en_US.UTF-8'

In [ ]:
!git clone https://github.com/imClumsyPanda/langchain-ChatGLM.git
%cd langchain-ChatGLM/

In [ ]:
#book download
!wget https://github.com/d2l-ai/d2l-zh/releases/download/v2.0.0/d2l-zh-pytorch-2.0.0.pdf

In [ ]:
#增加依赖
%%writefile requirements.txt
pymupdf
paddlepaddle==2.4.2
paddleocr
langchain==0.0.146
transformers==4.27.1
unstructured[local-inference]
layoutparser[layoutmodels,tesseract]
nltk
sentence-transformers
beautifulsoup4
icetk
cpm_kernels
faiss-cpu
accelerate
gradio==3.28.3
fastapi
uvicorn
peft
pypinyin
bitsandbytes
protobuf==3.20.*
#protobuf~=3.19.0
#detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2


Overwriting requirements.txt


In [ ]:
!cat requirements.txt

In [ ]:
# 安装依赖
!pip install -r requirements.txt -i https://mirror.sjtu.edu.cn/pypi/web/simple

In [ ]:
#安装detectron2 支持pdf
!git clone https://github.com/facebookresearch/detectron2.git
%cd detectron2
!pip install -e .

In [ ]:
!pip list|grep detectron2

detectron2                    0.6                  /content/langchain-ChatGLM/detectron2


In [ ]:
#回到代码路径
!pwd
%cd /content/langchain-ChatGLM
!ls -lh

/content/langchain-ChatGLM/detectron2
/content/langchain-ChatGLM
total 27M
drwxr-xr-x  2 root root 4.0K May 16 06:14 agent
-rw-r--r--  1 root root  15K May 16 06:14 api.py
drwxr-xr-x  3 root root 4.0K May 16 06:14 chains
-rw-r--r--  1 root root   27 May 16 06:14 cli.bat
-rw-r--r--  1 root root 2.0K May 16 06:14 cli_demo.py
-rw-r--r--  1 root root 1.2K May 16 06:14 cli.py
-rw-r--r--  1 root root   31 May 16 06:14 cli.sh
drwxr-xr-x  2 root root 4.0K May 16 06:14 configs
drwxr-xr-x  3 root root 4.0K May 16 06:14 content
-rw-r--r--  1 root root 1.7K May 16 06:14 CONTRIBUTING.md
-rw-r--r--  1 root root  27M Dec  8 00:57 d2l-zh-pytorch-2.0.0.pdf
drwxr-xr-x 17 root root 4.0K May 16 06:20 detectron2
-rw-r--r--  1 root root  868 May 16 06:14 Dockerfile
-rw-r--r--  1 root root  462 May 16 06:14 Dockerfile-cuda
drwxr-xr-x  2 root root 4.0K May 16 06:14 docs
drwxr-xr-x  2 root root 4.0K May 16 06:14 img
-rw-r--r--  1 root root  12K May 16 06:14 LICENSE
drwxr-xr-x  2 root root 4.0K May 16 06:14 loa

In [ ]:
# 使用 Markdown 格式打印模型输出
from IPython.display import display, Markdown, clear_output

def display_answer(agent, query, vs_path, history=[]):
    for resp, history in local_doc_qa.get_knowledge_based_answer(query=query,
                                                                 vs_path=vs_path,
                                                                 chat_history=history,
                                                                 streaming=True):
        clear_output(wait=True)
        display(Markdown(resp["result"]))
    return resp, history

In [ ]:
%cd /usr/local/lib/python3.10/dist-packages/paddleocr/tools

/usr/local/lib/python3.10/dist-packages/paddleocr/tools


In [ ]:
!touch __init__.py

In [ ]:
#回到代码路径
!pwd
%cd /content/langchain-ChatGLM
!ls -l

In [ ]:
!pip install --ignore-installed Pillow==9.0.0

In [ ]:
import torch.cuda
import torch.backends
# from .configs.model_config import *

from configs import model_config

# 全局参数，修改后请重新初始化
model_config.embedding_model_dict = {
    "ernie-tiny": "nghuyong/ernie-3.0-nano-zh",
    "ernie-base": "nghuyong/ernie-3.0-base-zh",
    "text2vec-base": "shibing624/text2vec-base-chinese",
    # "text2vec": "/home/mw/input/text2vec2538",
}
model_config.llm_model_dict = {
    "chatyuan": "ClueAI/ChatYuan-large-v2",
    "chatglm-6b-int4-qe": "THUDM/chatglm-6b-int4-qe",
    "chatglm-6b-int4": "THUDM/chatglm-6b-int4",
    "chatglm-6b-int8": "THUDM/chatglm-6b-int8",
    # "chatglm-6b": "/home/mw/input/ChatGLM6B6449",
}
model_config.VS_ROOT_PATH = "/content/temp"

from chains.local_doc_qa import LocalDocQA

EMBEDDING_MODEL = "text2vec-base" # embedding 模型，对应 embedding_model_dict
VECTOR_SEARCH_TOP_K = 6
LLM_MODEL = "chatglm-6b-int4"     # LLM 模型名，对应 llm_model_dict
LLM_HISTORY_LEN = 3
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

local_doc_qa = LocalDocQA()

local_doc_qa.init_cfg(llm_model=LLM_MODEL,
                          embedding_model=EMBEDDING_MODEL,
                          llm_history_len=LLM_HISTORY_LEN,
                          top_k=VECTOR_SEARCH_TOP_K)

INFO:root:
loading model config
llm device: cuda
embedding device: cuda
dir: /content/langchain-ChatGLM
flagging username: 7579abb371514279be91915a44897724

INFO:numexpr.utils:NumExpr defaulting to 2 threads.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c
Compiling gcc -O3 -fPIC -std=c99 /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.c -shared -o /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/THUDM/chatglm-6b-int4/02a065cf2797029c036a02cac30f1da1a9bc49a3/quantization_kernels.so
Using quantization cache
Applying quantization to glm layers


In [ ]:
vs_path, _ = local_doc_qa.init_knowledge_vector_store("d2l-zh-pytorch-2.0.0.pdf")
vs_path

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="什么是 DenseNet 模型", vs_path=vs_path, history=history)
resp["source_documents"]

DenseNet 模型是一种基于深度学习的计算机视觉模型，由 Google 于 2016 年提出。该模型在图像分类任务中表现出色，被广泛应用于计算机视觉领域。

DenseNet 模型主要由两个主要部分组成：稠密块和过渡层。稠密块用于处理图像的特征，过渡层则用于控制模型的维数，从而进一步减少通道的数量。这两个部分通过组合使用来实现模型的训练和评估。

在 DenseNet 模型中，稠密块采用平均汇聚层，过渡层则采用平均汇聚层和最大汇聚层的组合。这种组合使用的目的是通过减少模型的维度来控制模型的复杂度，从而提高模型在图像分类任务中的性能。

DenseNet 模型的优点之一是其模型参数比 ResNet 小，原因是 DenseNet 过渡层使用平均汇聚层，而不是最大汇聚层。另一个优点则是其训练速度更快，因为 DenseNet 模型中减少了通道的数量，从而加速了模型的训练。

DenseNet 模型是深度学习领域中非常重要的一种模型，被广泛应用于计算机视觉领域，包括图像分类、目标检测和图像分割等任务。

[Document(page_content=' . . . . . . 289 7.7.3 过渡层 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 290 7.7.4 DenseNet模型 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 291 7.7.5 训练模型 . . . . . . . . . . . . . .', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 170}),
 Document(page_content=' blk = transition_block(23, 10) blk(Y).shape torch.Size([4, 10, 4, 4]) 7.7.4 DenseNet模型 我们来构造DenseNet模型。DenseNet⾸先使⽤同ResNet⼀样的单卷积层和最⼤汇聚层。 b1 = nn.Sequential( nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), nn.BatchNorm2d(64), nn.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 118}),
 Document(page_content=' • DenseNet的主要构建模块是稠密块和过渡层。 • 在构建DenseNet时，我们需要通过添加过渡层来控制⽹络的维数，从⽽再次减少通道的数量。 292 7. 现代卷积神经⽹络  练习 1. 为什么我们在过渡层使⽤平均汇聚层⽽不是最⼤汇聚层？ 2. DenseNet的优点之⼀是其模型参数⽐ResNet⼩。为什么呢？ 3. DenseNet⼀个诟病的问题是内存或显存消耗过多。 1. 真的是这样吗？可以把输⼊形状换成224 × 224，来看看实际的显存消耗。 2.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 167}),
 Docume

In [ ]:
# 多轮对话
resp, history = display_answer(local_doc_qa, query="它与ResNet的关键区别是什么？", vs_path=vs_path, history=history)
resp["source_documents"]

稠密连接网络(DenseNet)与ResNet的关键区别在于输出的是连接而不是简单的相加。在稠密连接网络中，使用泰勒展开式(Taylor expansion)将函数分解成越来越高阶的项，其中连接项的权重随着层数的增加而增加，从而实现更好的性能和更强的网络结构。在ResNet中，每个模块有4个卷积层，加上第1个7 × 7卷积层和最后一个全连接层，共有18层，因此其网络结构更加复杂，但ResNet的模块里的残差块数可以配置不同，使得网络结构可以更加灵活。因此，稠密连接网络(DenseNet)在性能和结构方面均优于ResNet。

[Document(page_content='  你的代码是否包含启发式设计选择？  2. 你遇到的哪些问题有许多解决它们的样本，但没有具体的⾃动化⽅法？这些可能是使⽤深度学习的主  要候选者。  3. 如果把⼈⼯智能的发展看作⼀场新的⼯业⾰命，那么算法和数据之间的关系是什么？它类似于蒸汽机  和煤吗？根本区别是什么？  4. 你还可以在哪⾥应⽤端到端的训练⽅法，⽐如 图1.1.2 、物理、⼯程和计量经济学？  Discussions35  35 https://discuss.d2l.ai/t/1744  38  1.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 208}),
 Document(page_content='Linear(512, 10)) 每个模块有4个卷积层（不包括恒等映射的1 × 1卷积层）。加上第⼀个7 × 7卷积层和最后⼀个全连接层，共 有18层。因此，这种模型通常被称为ResNet-18。通过配置不同的通道数和模块⾥的残差块数可以得到不同 的ResNet模型，例如更深的含152层的ResNet-152。虽然ResNet的主体架构跟GoogLeNet类似，但ResNet架 构更简单，修改也更⽅便。这些因素都导致了ResNet迅速被⼴泛使⽤。图7.6.4描述了完整的ResNet-18。 7.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 196}),
 Document(page_content=' 为什么即使函数类是嵌套的，我们仍然要限制增加函数的复杂性呢？ Discussions96 7.7 稠密连接⽹络（DenseNet） ResNet极⼤地改变了如何参数化深层⽹络中函数的观点。稠密连接⽹络（DenseNet）(Huang et al., 2017)在 某种程度上是ResNet的逻辑扩展。让我们先从数学上了解⼀下。 7.7.1 从ResNet到DenseNet 回想⼀下任意函数的泰勒展开式（Taylor expansion），它把这个函数分解成越来越⾼阶的项。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 203}),

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="梯度是什么", vs_path=vs_path, history=history)
resp["source_documents"]

梯度是一种用于计算模型参数更新的方法。在深度学习中，梯度下降是非常重要的优化算法，通过不断地在损失函数递减的方向上更新参数来降低误差。

在梯度下降中，我们假设损失函数(数据集中所有样本的损失均值)与模型参数(即向量表示的目标函数的梯度)之间有线性关系，通过梯度向量来更新模型参数。计算损失函数对模型参数的梯度(称为梯度)是梯度下降算法的核心，可以通过计算损失函数对模型参数的导数(称为梯度)来完成。

在实际应用中，为了加速计算，我们通常会在每次需要计算更新的时候随机抽取一小部分样本，这种变体称为小批量随机梯度下降(minibatch stochastic gradient descent)。

在计算损失函数对模型参数的梯度时，我们需要考虑噪声模型中的指数分布。在这种情况下，我们通常会将梯度设置为0，并使用矩阵方程求解来找到解析解。

[Document(page_content=' 它通过不断地在损失函数递减的⽅向上更新参数来降低误差。 梯度下降最简单的⽤法是计算损失函数（数据集中所有样本的损失均值）关于模型参数的导数（在这⾥也可 以称为梯度）。但实际中的执⾏可能会⾮常慢：因为在每⼀次更新参数之前，我们必须遍历整个数据集。因此， 我们通常会在每次需要计算更新的时候随机抽取⼀⼩批样本，这种变体叫做⼩批量随机梯度下降（minibatch stochastic gradient descent）。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 220}),
 Document(page_content='为了简化问题，可以忽略偏置b（我们可以通过 向X添加所有值为1的⼀列来做到这⼀点）。 1. ⽤矩阵和向量表⽰法写出优化问题（将所有数据视为单个矩阵，将所有⽬标值视为单个向量）。 2. 计算损失对w的梯度。 3. 通过将梯度设为0、求解矩阵⽅程来找到解析解。 4. 什么时候可能⽐使⽤随机梯度下降更好？这种⽅法何时会失效？ 3. 假定控制附加噪声ϵ的噪声模型是指数分布。也就是说，p(ϵ) = 1 2 exp(−|ϵ|) 1. 写出模型− log P(y | X)下数据的负对数似然。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 210}),
 Document(page_content=' True, True, True]) 这个例⼦表明第三个和第五个神经⽹络层的参数是绑定的。它们不仅值相等，⽽且由相同的张量表⽰。因此， 如果我们改变其中⼀个参数，另⼀个参数也会改变。这⾥有⼀个问题：当参数绑定时，梯度会发⽣什么情况？ 答案是由于模型参数包含梯度，因此在反向传播期间第⼆个隐藏层（即第三个神经⽹络层）和第三个隐藏层 （即第五个神经⽹络层）的梯度会加在⼀起。 ⼩结 • 我们有⼏种⽅法可以访问、初始化和绑定模型参数。 • 我们可以使⽤⾃定义初始化⽅法。 练习 1. 使⽤ 5.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 217}),
 Document(page_content='d2l def 

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="softmax回归怎么理解", vs_path=vs_path, history=history)
resp["source_documents"]

softmax回归是一种常用的神经网络模型，用于对给定的数据进行分类。在训练时，我们使用数据样本的权重矩阵(也称为权重向量)来预测数据样本的标签。在预测时，我们使用每个数据样本的权重向量来计算预测标签，并将这些标签的值拼接起来得到最终的分类输出。

具体来说，softmax回归的模型架构是一个由3个全连接层组成的神经网络。在每个全连接层中，我们使用权重向量表示输入特征，并使用偏置向量表示对特征的线性变换。最终，我们将每个输入特征的权重向量和偏置向量的值相加得到输出向量，并将这些向量拼接起来得到最终的分类输出。

由于softmax函数的非线性特性，我们通常使用softmax函数作为模型的输出函数，而不是直接使用输出向量进行分类。这样可以保留模型的非线性特性，同时简化模型的结构。

在实际应用中，softmax回归经常用于分类问题。例如，我们可能需要对图像中的物体进行分类，或者对文本数据进行分类。通过使用softmax回归模型，我们可以对给定的数据样本进行分类，并使用模型的预测结果进行进一步的处理，例如生成模型。

[Document(page_content=' . . . . . . . 104 3.3.7 训练 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 105 3.4 softmax回归 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 106 3.4.1 分类问题 . . . . . . . . .', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 159}),
 Document(page_content=' . . . . . 121 3.6.7 预测 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 124 3.7 softmax回归的简洁实现 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 125 3.7.1 初始化模型参数 . . . . . . . . . . . . .', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 154}),
 Document(page_content='由此，我们已经将所有权重放到⼀个3 × 4矩阵中。对于给定数据样本的特征x，我们 的输出是由权重与输⼊特征进⾏矩阵-向量乘法再加上偏置b得到的。 3.4. softmax回归 107  3.4.3 全连接层的参数开销 正如我们将在后续章节中看到的，在深度学习中，全连接层⽆处不在。然⽽，顾名思义，全连接层是“完全”连 接的，可能有很多可学习的参数。具体来说，对于任何具有d个输⼊和q个输出的全连接层，参数开销为O(dq)， 这个数字在实践中可能⾼得令⼈望⽽却步。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 143

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="前向传播、反向传播和计算图怎么理解，有啥区别", vs_path=vs_path, history=history)
resp["source_documents"]

前向传播和反向传播是多层感知机(多层感知机是一种用于图像和视频分类的神经网络模型)中的重要概念，用于计算和存储中间变量和参数的梯度。

前向传播是指在计算图中按顺序计算和存储中间变量，它的顺序是从输入层到输出层。前向传播的工作原理是，先按顺序执行每个操作，然后根据链式法则计算梯度，最后将这些梯度按顺序存储到变量中。这个过程可以通过绘制计算图来可视化。

反向传播按相反的顺序(从输出层到输入层)计算和存储神经网络的中间变量和参数的梯度。反向传播的工作原理是，先执行每个操作，然后根据梯度下降算法计算梯度，最后将这些梯度按顺序存储到变量中。这个过程可以通过绘制计算图来可视化。

前向传播和反向传播在训练深度学习模型时是相互依赖的。前向传播和反向传播的计算和存储过程都需要大量的内存，因此需要在训练模型时进行合理的内存管理和优化。

前向传播和反向传播是神经网络中重要的概念，它们的计算和存储过程都需要大量的内存，因此在训练深度学习模型时需要进行合理的内存管理和优化。

[Document(page_content=' . . . . . . . 160 4.6.5 简洁实现 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 163 4.7 前向传播、反向传播和计算图 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 165 4.7.1 前向传播 . . . . . . . . . . . . . . . . .', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 160}),
 Document(page_content=' (4.7.6) 在下⾯的讨论中，我们将J称为⽬标函数（objective function）。 67 https://discuss.d2l.ai/t/1813 4.7. 前向传播、反向传播和计算图 165  4.7.2 前向传播计算图 绘制计算图有助于我们可视化计算中操作符和变量的依赖关系。图4.7.1 是与上述简单⽹络相对应的计算图， 其中正⽅形表⽰变量，圆圈表⽰操作符。左下⻆表⽰输⼊，右上⻆表⽰输出。注意显⽰数据流的箭头⽅向主 要是向右和向上的。 图4.7.1: 前向传播的计算图 4.7.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 163}),
 Document(page_content='此外，这些中间值的⼤⼩与⽹络层的数量和批量的⼤⼩⼤致成正⽐。因此，使⽤更⼤的批量来训练更深 层次的⽹络更容易导致内存不⾜（out of memory）错误。 ⼩结 • 前向传播在神经⽹络定义的计算图中按顺序计算和存储中间变量，它的顺序是从输⼊层到输出层。 • 反向传播按相反的顺序（从输出层到输⼊层）计算和存储神经⽹络的中间变量和参数的梯度。 • 在训练深度学习模型时，前向传播和反向传播是相互依赖的。 • 训练⽐预测需要更多的内存。 4.7. 前向传播、反向传播和计算图 167  练习 1. 假设⼀些标量函数X的输⼊X是n × m矩阵。f相对于X的梯度维数是多少？ 2. 向本节中描

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="什么全连接层，有什么作用", vs_path=vs_path, history=history)
resp["source_documents"]

全连接层(fully connected layer)是神经网络中的一个重要组成部分，作用是将输入的序列数据映射到输出序列数据。在深度学习中，全连接层被广泛应用于图像、语音等图像处理任务，也被用于自然语言处理中的语音识别、机器翻译等任务。

全连接层的作用是将输入的数据通过矩阵向量乘法得到输出，也就是将输入和输出之间的关系表示为矩阵向量之间的乘积。这个过程可以看作是将输入序列映射到输出序列的过程，从而得到模型的输出。全连接层的作用就是将输入序列映射到输出序列，并且在这个过程中可以学习到一些可学习的参数。

全连接层在深度学习中广泛应用，但并不是唯一的模型结构。在深度学习中，除了全连接层外，还有其他模型结构，如卷积神经网络(CNN)和循环神经网络(RNN)等。这些模型结构在深度学习领域中有着广泛的应用，并且在不同类型的任务中可以发挥不同的作用。

[Document(page_content=' 对于线性回归，每个输⼊都与每个输出（在本例中只有⼀个输出）相连，我们将这种变换（图3.1.2中的输出 层）称为全连接层（fully-connected layer）或称为稠密层（dense layer）。下⼀章将详细讨论由这些层组成 的⽹络。 ⽣物学 线性回归发明的时间（1795年）早于计算神经科学，所以将线性回归描述为神经⽹络似乎不合适。当控制学 家、神经⽣物学家沃伦·⻨库洛奇和沃尔特·⽪茨开始开发⼈⼯神经元模型时，他们为什么将线性模型作为⼀ 个起点呢？', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 190}),
 Document(page_content='  回顾 图3.1.2中的单层⽹络架构，这⼀单层被称为全连接层（fully connected layer），因为它的每⼀个输⼊都  通过矩阵 向量乘法得到它的每个输出。  在PyTorch中，全连接层在Linear类中定义。值得注意的是，我们将两个参数传递到nn.Linear中。第⼀个指  定输⼊特征形状，即2，第⼆个指定输出特征形状，输出特征形状为单个标量，因此为1。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 173}),
 Document(page_content=' softmax回归 107  3.4.3 全连接层的参数开销 正如我们将在后续章节中看到的，在深度学习中，全连接层⽆处不在。然⽽，顾名思义，全连接层是“完全”连 接的，可能有很多可学习的参数。具体来说，对于任何具有d个输⼊和q个输出的全连接层，参数开销为O(dq)， 这个数字在实践中可能⾼得令⼈望⽽却步。幸运的是，将d个输⼊转换为q个输出的成本可以减少到O( dq n )，其 中超参数n可以由我们灵活指定，以在实际应⽤中平衡参数节约和模型有效性 (Zhang et al., 2021)。 3.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 179}),
 Document(page_content='1.1: ⼀个单隐藏层的多层感知机，具有5个隐藏单元 这个多层感知机有

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="什么LSTM，有什么作用", vs_path=vs_path, history=history)
resp["source_documents"]

LSTM 是长短时记忆网络(Long Short-Term Memory,LSTM)的缩写，它是一种用于序列建模的深度学习模型，能够处理长期依赖关系。LSTM 模型通过记忆单元来捕捉序列中的长期依赖关系，使得模型能够更好地预测未来数据。

LSTM 模型由三个门控单元组成，分别是输入门、遗忘门和输出门。输入门用于控制信息的输入和输出，遗忘门用于控制信息的处理和遗忘，而输出门用于控制信息的的输出和输出。这 三个门控单元相互交互作用，使得LSTM 模型能够有效地捕捉到序列中的长期依赖关系。

LSTM 模型的输出被组织成一张图，称为状态图，表示模型当前所处的状态，以及从当前状态到 未来状态的转移。状态图通过状态门和遗忘门来组织信息，使得模型能够更好地预测未来数据。

LSTM 模型在自然语言处理领域有着广泛的应用，例如文本分类、机器翻译、情感分析等。在 NLP 任务中，LSTM 模型通常被用来对序列数据进行分类和预测，能够很好地捕捉到序列数据 中的长期依赖关系。

[Document(page_content='html 52 https://d2l.ai/chapter_appendix-mathematics-for-deep-learning/information-theory.html 110 3. 线性神经⽹络  信息量 压缩与预测有什么关系呢？想象⼀下，我们有⼀个要压缩的数据流。如果我们很容易预测下⼀个数据，那么 这个数据就很容易压缩。为什么呢？举⼀个极端的例⼦，假如数据流中的每个数据完全相同，这会是⼀个⾮ 常⽆聊的数据流。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 262}),
 Document(page_content='shape[0], device=device) else: if isinstance(net, nn.Module) and not isinstance(state, tuple): # state对于nn.GRU是个张量 state.detach_() else: # state对于nn.LSTM或对于我们从零开始实现的模型是个张量 for s in state: s.detach_() y = Y.T.reshape(-1) X, y = X.to(device), y.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 266}),
 Document(page_content=' 例如，循环神经⽹络在实践中⼀个常⻅问题是数值不稳定性。尽管我们已经应⽤了梯度裁剪等技巧来缓解这 个问题，但是仍需要通过设计更复杂的序列模型来进⼀步处理它。具体来说，我们将引⼊两个⼴泛使⽤的⽹ 络，即⻔控循环单元（gated recurrent units，GRU）和 ⻓短期记忆⽹络（long short-term memory，LSTM）。 然后，我们将基于⼀个单向隐藏层来扩展循环神经⽹络架构。我们将描述具有多个隐藏层的深层架构，并讨 论基于前向和后向循环计算的双向设计。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 279}),
 Document(page_content='zero

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="什么Transformer，有什么作用", vs_path=vs_path, history=history)
resp["source_documents"]

Transformer是一种用于序列到序列模型训练的数据增强技术。它由编码器和解码器组成，编码器使用多头自注意力机制来提取序列中的长程依赖关系，而解码器则使用位置编码来重构序列。Transformer架构在自然语言处理任务中取得了很好的效果，例如机器翻译、文本摘要和语音识别等。

[Document(page_content=" 'SoftmaxTransform', �→'SoftplusTransform', 'StackTransform', 'StickBreakingTransform', 'StudentT', 'TanhTransform', �→'Transform', 'TransformedDistribution', 'Uniform', 'VonMises', 'Weibull', 'Wishart', '__all__', '__ �→builtins__', '__cached__',", metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 260}),
 Document(page_content=" �→'register_kl', 'relaxed_bernoulli', 'relaxed_categorical', 'studentT', 'transform_to', 'transformed_ �→distribution', 'transforms', 'uniform', 'utils', 'von_mises', 'weibull', 'wishart'] 通常可以忽略以“__”（双下划线）开始和结束的函数，它们是Python中的特殊对象，", metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 263}),
 Document(page_content='然后将其加载到内存中""" trans = [transforms.ToTensor()] if resize: trans.insert(0, transforms.Resize(resize)) trans = transforms.Compose(trans) mnist_train = torchvision.datasets.FashionMNIST( root=".', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 226}),
 Document(page_content=' 10.7.1 模型 Transformer作为编码器－解码器架构的

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="注意⼒机制怎么理解，多头注意力如何理解，两者有啥不同点", vs_path=vs_path, history=history)
resp["source_documents"]

注意力机制是深度学习中的重要概念，用于将输入的信息组织成有意义的输出。在深度学习中，注意力机制被广泛应用于Transformer架构中，该架构中使用了多头注意力机制。

多头注意力机制是一种注意力机制，它将查询、键和值分解为多个子空间，每个子空间对应一个头。通过多头注意力机制，模型可以学习不同子空间之间的关系，从而更好地处理输入数据。

在Transformer中，多头注意力机制通过全连接层来实现可学习的线性变换。在实现多头注意力之前，需要学习多个线性投影，这些投影用于将查询、键和值分解为多个子空间，然后这些子空间将并行地送到注意力汇聚中。最终，这些子空间的输出拼接在一起，并通过另一个可以学习的线性投影进行变换，以产生最终输出。

多头注意力和自注意力机制是深度学习中非常重要的机制，它们在处理输入数据时都有不同的效果和优点。多头注意力机制可以更好地捕捉不同子空间之间的关系，而自注意力机制可以更好地捕捉查询和键之间的交互关系。这两种机制在处理不同的问题和任务时有不同的应用。

[Document(page_content='具体来说，我们将展⽰如何  使⽤这些函数来设计Bahdanau注意⼒。Bahdanau注意⼒是深度学习中的具有突破性价值的注意⼒模型，它  双向对⻬并且可以微分。  最后将描述仅仅基于注意⼒机制的Transformer架构，该架构中使⽤了多头注意⼒（multi head attention）和  ⾃注意⼒（self attention）。⾃2017年横空出世，Transformer⼀直都普遍存在于现代的深度学习应⽤中，例  如语⾔、视觉、语⾳和强化学习领域。  391  10.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 163}),
 Document(page_content=' 为此，与其只使⽤单独⼀个注意⼒汇聚，我们可以⽤独⽴学习得到的h组不同的 线性投影（linear projections） 来变换查询、键和值。然后，这h组变换后的查询、键和值将并⾏地送到注意⼒汇聚中。最后，将这h个注意 ⼒汇聚的输出拼接在⼀起，并且通过另⼀个可以学习的线性投影进⾏变换，以产⽣最终输出。这种设计被称 为多头注意⼒（multihead attention）(Vaswani et al., 2017)。对于h个注意⼒汇聚输出，每⼀个注意⼒汇聚都 被称作⼀个头（head）。图10.5.1 展⽰了使⽤全连接层来实现可学习的线性变换的多头注意⼒。 图10.5.1: 多头注意⼒：多个头连结然后线性变换 10.5.1 模型 在实现多头注意⼒之前，让我们⽤数学语⾔将这个模型形式化地描述出来。给定查询q ∈ Rdq、键k ∈ Rdk和 值v ∈ Rdv，每个注意⼒头hi（i = 1, . . . , h）的计算⽅法为： hi = f(W(q) i q, W(k) i k, W(v) i v) ∈ Rpv, (10.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 172}),
 Document(page_content='3节中的加性注意⼒和缩放点积注意⼒。多头注意⼒的输出需要经过另⼀个线性转换，它对应着h个 121 https://discuss.d2l.ai/t/5754 10.5. 多头注意⼒

In [ ]:
# 可以额外返回参照的知识库源信息
history = []
resp, history = display_answer(local_doc_qa, query="词嵌⼊有哪些种类，各种的实现原理分别是啥，有啥优势", vs_path=vs_path, history=history)
resp["source_documents"]

根据已知信息，回答用户的问题：

1. 词嵌入的种类：

词嵌入有两种主要类型：词向量嵌入和端到端嵌入。

词向量嵌入是一种基于向量场理论的技术，将单词映射到实向量空间，其中每个单词对应一个向量。词向量嵌入被广泛应用于文本分类、情感分析、命名实体识别等任务。

端到端嵌入是一种将词嵌入嵌入到循环神经网络中的方法，可以更有效地捕捉单词之间的关系。端到端嵌入在自然语言处理中的应用场景包括文本分类、机器翻译、情感分析等。

2. 各种词嵌入的实现原理：

2.1 词向量嵌入：

词向量嵌入的实现原理是将单词映射到实向量空间，每个单词对应一个向量。实现方式包括词嵌入模型(如 Word2Vec、词向量等)、词嵌入模型嵌入到循环神经网络中(如 WordNet 模型)。词向量嵌入的优点是可以捕捉单词之间的关系，提高模型的性能，但是需要大量的训练数据和计算资源。

2.2 端到端嵌入：

端到端嵌入是将词嵌入嵌入到循环神经网络中的方法。该方法可以更有效地捕捉单词之间的关系，提高模型的性能，但是需要大量的训练数据和计算资源。端到端嵌入的优点是可以更好地处理复杂的任务，如文本分类、机器翻译、情感分析等，但是需要更多的训练数据和计算资源。

3. 优势：

词嵌入技术可以更好地捕捉单词之间的关系，提高模型的性能，适用于文本分类、机器翻译、情感分析等复杂的任务。词嵌入技术需要大量的训练数据和计算资源，适用于大型数据集，如维基百科、维基百科知识图谱等。

[Document(page_content='例如，某⽤⼾跳过播放列表中的某些歌曲，这可能说明这些歌曲  对此⽤⼾不⼤合适。总的来说，推荐系统会为“给定⽤⼾和物品”的匹配性打分，这个“分数”可能是估计  的评级或购买的概率。由此，对于任何给定的⽤⼾，推荐系统都可以检索得分最⾼的对象集，然后将其推荐  给⽤⼾。以上只是简单的算法，⽽⼯业⽣产的推荐系统要先进得多，它会将详细的⽤⼾活动和项⽬特征考虑  在内。推荐系统算法经过调整，可以捕捉⼀个⼈的偏好。⽐如，图1.3.4 是亚⻢逊基于个性化算法推荐的深度  学习书籍，成功地捕捉了作者的喜好。', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 175}),
 Document(page_content=' 循环神经⽹络  8.6 循环神经⽹络的简洁实现 虽然 8.5节 对了解循环神经⽹络的实现⽅式具有指导意义，但并不⽅便。本节将展⽰如何使⽤深度学习框架 的⾼级API提供的函数更有效地实现相同的语⾔模型。我们仍然从读取时光机器数据集开始。 import torch from torch import nn from torch.nn import functional as F from d2l import torch as d2l batch_size, num_steps = 32,', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score': 177}),
 Document(page_content='如果我们可以⾃动确定η，或者完全不必选择学习 率，会怎么样？除了考虑⽬标函数的值和梯度、还考虑它的曲率的⼆阶⽅法可以帮我们解决这个问题。虽然 由于计算代价的原因，这些⽅法不能直接应⽤于深度学习，但它们为如何设计⾼级优化算法提供了有⽤的思 维直觉，这些算法可以模拟下⾯概述的算法的许多理想特性。 ⽜顿法 回顾⼀些函数f : Rd → R的泰勒展开式，事实上我们可以把它写成 f(x + ϵ) = f(x) + ϵ⊤∇f(x) + 1 2ϵ⊤∇2f(x)ϵ + O(∥ϵ∥3). (11.3.', metadata={'source': 'd2l-zh-pytorch-2.0.0.pdf', 'score